In [7]:
import psycopg2
import os

In [8]:
# Informações de conexão com o PostgreSQL
DB_HOST = "airflow-postgres"
DB_PORT = "5432"
DB_USER = os.getenv('POSTGRES_USER')
DB_PASSWORD = os.getenv('POSTGRES_PASSWORD')
DB_NAME = "azurecost"

conn = None

In [9]:
try:
    # Conecta-se ao banco de dados 'postgres' para criar o banco 'azurecost'
    conn = psycopg2.connect(
        host=DB_HOST,
        port=DB_PORT,
        database="postgres",
        user=DB_USER,
        password=DB_PASSWORD
    )
    conn.autocommit = True
    cur = conn.cursor()

    # Verifica e cria o banco de dados 'azurecost' se não existir
    cur.execute(f"SELECT 1 FROM pg_database WHERE datname='{DB_NAME}'")
    exists = cur.fetchone()
    if not exists:
        cur.execute(f"CREATE DATABASE {DB_NAME}")
        print(f"Banco de dados '{DB_NAME}' criado com sucesso.")
    else:
        print(f"Banco de dados '{DB_NAME}' já existe.")

    cur.close()
    conn.close()  # Fecha a conexão com 'postgres'

    # Conecta-se ao banco de dados 'azurecost' para criar a tabela
    conn = psycopg2.connect(
        host=DB_HOST,
        port=DB_PORT,
        database=DB_NAME,
        user=DB_USER,
        password=DB_PASSWORD
    )
    conn.autocommit = True
    cur = conn.cursor()

    # Comando SQL para criar a tabela 'azure_cost_data'
    create_table_query = """
    CREATE TABLE IF NOT EXISTS azure_cost_data (
        ResourceName VARCHAR,
        SubscriptionId VARCHAR,
        ResourceGroup VARCHAR,
        Provider VARCHAR,
        StatusRecourse VARCHAR,
        PreTaxCost DOUBLE PRECISION,
        Pct_Change DOUBLE PRECISION,
        Currency VARCHAR,
        UsageDate DATE,
        TendenciaCusto VARCHAR,
        PrevisaoProxima DOUBLE PRECISION
    );
    """

    cur.execute(create_table_query)
    conn.commit()
    print(f"Tabela 'azure_cost_data' criada com sucesso no banco de dados '{DB_NAME}'.")

    cur.close()

except psycopg2.Error as e:
    print(f"Erro ao conectar ou criar o banco de dados/tabela: {e}")
finally:
    if conn:
        conn.close()

Banco de dados 'azurecost' criado com sucesso.
Tabela 'azure_cost_data' criada com sucesso no banco de dados 'azurecost'.


In [10]:
from pyspark.sql import SparkSession
from delta.tables import DeltaTable
import os

DELTA_LAKE_PACKAGE = "io.delta:delta-core_2.12:3.3.2"

spark = SparkSession.builder \
    .appName("PySpark Delta Lake MinIO Save") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .config("spark.hadoop.fs.s3a.endpoint", "http://minio:9000") \
    .config("spark.hadoop.fs.s3a.access.key", os.getenv("KEY_ACCESS")) \
    .config("spark.hadoop.fs.s3a.secret.key", os.getenv("KEY_SECRETS")) \
    .config("spark.hadoop.fs.s3a.path.style.access", "true") \
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
    .config("spark.hadoop.fs.s3a.aws.credentials.provider", "org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider") \
    .getOrCreate()


# Caminho para a tabela Delta (no seu MinIO)
gold_path = "s3a://azurecost/gold"

# Inicializa objeto DeltaTable
delta_table = DeltaTable.forPath(spark, gold_path)

# Obtém todos os valores únicos da partição
partitions_df = delta_table.toDF().select("data_ref").distinct()

# Obtém o valor mais recente da partição
max_partition = partitions_df.agg({"data_ref": "max"}).collect()[0][0]
print(f"Última partição disponível: {max_partition}")

# Lê os dados somente da última partição
df = spark.read.format("delta").load(gold_path).filter(f"data_ref = '{max_partition}'")

df = df.select(
    "ResourceName",
    "SubscriptionId",
    "ResourceGroup",
    "Provider",
    "StatusRecourse",
    "PreTaxCost",
    "Pct_Change",
    "Currency",
    "UsageDate",
    "TendenciaCusto",
    "PrevisaoProxima"
)

df.show(truncate=False)

Py4JJavaError: An error occurred while calling None.org.apache.spark.api.java.JavaSparkContext.
: org.apache.spark.SparkException: Only one SparkContext should be running in this JVM (see SPARK-2243).The currently running SparkContext was created at:
org.apache.spark.api.java.JavaSparkContext.<init>(JavaSparkContext.scala:58)
java.base/jdk.internal.reflect.NativeConstructorAccessorImpl.newInstance0(Native Method)
java.base/jdk.internal.reflect.NativeConstructorAccessorImpl.newInstance(NativeConstructorAccessorImpl.java:77)
java.base/jdk.internal.reflect.DelegatingConstructorAccessorImpl.newInstance(DelegatingConstructorAccessorImpl.java:45)
java.base/java.lang.reflect.Constructor.newInstanceWithCaller(Constructor.java:499)
java.base/java.lang.reflect.Constructor.newInstance(Constructor.java:480)
py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:247)
py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:374)
py4j.Gateway.invoke(Gateway.java:238)
py4j.commands.ConstructorCo

In [ ]:
jdbc_url = f"jdbc:postgresql://{DB_HOST}:{DB_PORT}/{DB_NAME}"

properties = {
    "user": DB_USER,
    "password": DB_PASSWORD,
    "driver": "org.postgresql.Driver"
}

df.write \
  .format("jdbc") \
  .option("url", jdbc_url) \
  .option("dbtable", "azure_cost_data") \
  .option("user", DB_USER) \
  .option("password", DB_PASSWORD) \
  .option("driver", "org.postgresql.Driver") \
  .mode("append") \
  .save()

print("Dados inseridos com sucesso na tabela 'azure_cost_data'.")  



Dados inseridos com sucesso na tabela 'azure_cost_data'.


In [11]:
import psycopg2
import os

DB_HOST = "airflow-postgres"
DB_PORT = "5432"
DB_USER = os.getenv('POSTGRES_USER')
DB_PASSWORD = os.getenv('POSTGRES_PASSWORD')

# Abre conexão com o banco padrão
conn = psycopg2.connect(
    host=DB_HOST,
    port=DB_PORT,
    user=DB_USER,
    password=DB_PASSWORD,
    database="postgres"
)
cur = conn.cursor()

# Executa a consulta para listar os bancos de dados
cur.execute("SELECT datname FROM pg_database;")
databases = cur.fetchall()

print("Bancos de dados disponíveis:")
for db in databases:
    print(db[0])

cur.close()
conn.close()

Bancos de dados disponíveis:
postgres
airflow
template1
template0
azurecost


In [13]:
import psycopg2
import os

DB_HOST = "airflow-postgres"
DB_PORT = "5432"
DB_USER = os.getenv('POSTGRES_USER')
DB_PASSWORD = os.getenv('POSTGRES_PASSWORD')
DB_NAME = "airflow"

conn = psycopg2.connect(
    host=DB_HOST,
    port=DB_PORT,
    user=DB_USER,
    password=DB_PASSWORD,
    database=DB_NAME
)

cur = conn.cursor()

# Query para listar tabelas do schema 'public'
cur.execute("""
    SELECT table_name 
    FROM information_schema.tables 
    WHERE table_schema = 'public'
    AND table_type = 'BASE TABLE';
""")

tables = cur.fetchall()

print(f"Tabelas no banco '{DB_NAME}':")
for table in tables:
    print(table[0])

cur.close()
conn.close()


Tabelas no banco 'airflow':
log
dag_priority_parsing_request
job
slot_pool
callback_request
dag_code
dag_pickle
ab_user
ab_register_user
connection
sla_miss
variable
import_error
serialized_dag
dataset_alias
dataset_alias_dataset
dataset
dataset_alias_dataset_event
dataset_event
dag_schedule_dataset_alias_reference
dag
dag_schedule_dataset_reference
task_outlet_dataset_reference
dataset_dag_run_queue
log_template
dag_run
dag_tag
dag_owner_attributes
ab_permission
ab_permission_view
ab_view_menu
ab_user_role
ab_role
dag_warning
dagrun_dataset_event
trigger
task_instance
dag_run_note
ab_permission_view_role
rendered_task_instance_fields
task_fail
task_map
task_reschedule
xcom
task_instance_note
task_instance_history
session
alembic_version


In [15]:
import psycopg2
import os

DB_HOST = "airflow-postgres"
DB_PORT = "5432"
DB_USER = os.getenv('POSTGRES_USER')
DB_PASSWORD = os.getenv('POSTGRES_PASSWORD')
DB_NAME = "airflow" 

table_name = 'dag'

conn = psycopg2.connect(
    host=DB_HOST,
    port=DB_PORT,
    user=DB_USER,
    password=DB_PASSWORD,
    database=DB_NAME
)

cur = conn.cursor()

cur.execute("""
    SELECT column_name, data_type, is_nullable, column_default
    FROM information_schema.columns
    WHERE table_schema = 'public'
    AND table_name = %s
    ORDER BY ordinal_position;
""", (table_name,))

columns = cur.fetchall()

print(f"Colunas da tabela '{table_name}':")
for col in columns:
    print(f"Nome: {col[0]}, Tipo: {col[1]}, Pode ser NULL: {col[2]}, Default: {col[3]}")

cur.close()
conn.close()


Colunas da tabela 'dag':
Nome: dag_id, Tipo: character varying, Pode ser NULL: NO, Default: None
Nome: root_dag_id, Tipo: character varying, Pode ser NULL: YES, Default: None
Nome: is_paused, Tipo: boolean, Pode ser NULL: YES, Default: None
Nome: is_subdag, Tipo: boolean, Pode ser NULL: YES, Default: None
Nome: is_active, Tipo: boolean, Pode ser NULL: YES, Default: None
Nome: last_parsed_time, Tipo: timestamp with time zone, Pode ser NULL: YES, Default: None
Nome: last_pickled, Tipo: timestamp with time zone, Pode ser NULL: YES, Default: None
Nome: last_expired, Tipo: timestamp with time zone, Pode ser NULL: YES, Default: None
Nome: scheduler_lock, Tipo: boolean, Pode ser NULL: YES, Default: None
Nome: pickle_id, Tipo: integer, Pode ser NULL: YES, Default: None
Nome: fileloc, Tipo: character varying, Pode ser NULL: YES, Default: None
Nome: processor_subdir, Tipo: character varying, Pode ser NULL: YES, Default: None
Nome: owners, Tipo: character varying, Pode ser NULL: YES, Default: Non

In [ ]:
try:
    conn = psycopg2.connect(
        host=DB_HOST,
        port=DB_PORT,
        database=DB_NAME,
        user=DB_USER,
        password=DB_PASSWORD
    )
    conn.autocommit = True
    cur = conn.cursor()

    drop_table_query = "DROP TABLE IF EXISTS azure_cost_data;"
    cur.execute(drop_table_query)
    print(f"Tabela 'azure_cost_data' excluída com sucesso.")

    cur.close()

except psycopg2.Error as e:
    print(f"Erro ao excluir a tabela: {e}")

finally:
    if conn:
        conn.close()

Tabela 'azure_cost_data' excluída com sucesso.


In [ ]:
try:
    conn = psycopg2.connect(
        host=DB_HOST,
        port=DB_PORT,
        database="postgres",  # banco diferente do que será dropado
        user=DB_USER,
        password=DB_PASSWORD
    )
    conn.autocommit = True
    cur = conn.cursor()

    # Fecha conexões ativas no banco azurecost
    cur.execute("""
        SELECT pg_terminate_backend(pid) 
        FROM pg_stat_activity 
        WHERE datname = %s;
    """, (DB_NAME,))

    # Exclui o banco azurecost
    cur.execute(f"DROP DATABASE IF EXISTS {DB_NAME};")
    print(f"Banco de dados '{DB_NAME}' excluído com sucesso.")

    cur.close()

except psycopg2.Error as e:
    print(f"Erro ao excluir o banco de dados: {e}")

finally:
    if conn:
        conn.close()


Banco de dados 'azurecost' excluído com sucesso.
